*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass

!pip install tensorflow-datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 578.3 MB 6.2 kB/s 
     |████████████████████████████████| 1.7 MB 48.8 MB/s 
     |████████████████████████████████| 5.8 MB 28.4 MB/s 
     |████████████████████████████████| 438 kB 65.6 MB/s 
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.5.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2022-07-13 18:23:23--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.03s   

2022-07-13 18:23:23 (11.6 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2022-07-13 18:23:23--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==========

In [ ]:
train_data = pd.read_csv(train_file_path, names=["type", "message"], sep="\t", header=None)
test_data = pd.read_csv(test_file_path, names=["type", "message"], sep="\t", header=None)

In [ ]:
train_data.head()

,type,message
0,ham,ahhhh just woken up had a bad dream about u ...
1,ham,you can never do nothing
2,ham,now u sound like manky scouse boy steve like ...
3,ham,mum say we wan to go then go then she can s...
4,ham,never y lei i v lazy got wat dat day ...


In [ ]:
train_data['message'] = train_data['message'].apply(lambda x: x.lower())
train_data['message'] = train_data['message'].apply(lambda x: re.sub('[^a-zA-z0-9\s]', ' ', x))

In [ ]:
tokenizer = Tokenizer(num_words=10000, split=' ')
tokenizer.fit_on_texts(train_data['message'].values)

In [ ]:
X = tokenizer.texts_to_sequences(train_data['message'].values)
X = pad_sequences(X, maxlen=190)

In [ ]:
Y = pd.factorize(train_data["type"])[0].astype('float32')

In [ ]:
model = keras.Sequential()

model.add(keras.layers.Embedding(10000, 128, input_length=X.shape[1]))
model.add(keras.layers.LSTM(196, dropout=0.2, recurrent_dropout=0.2))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.compile(optimizer="adam", metrics=['accuracy'], loss='binary_crossentropy')

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 190, 128)          1280000   
                                                                 
 lstm_1 (LSTM)               (None, 196)               254800    
                                                                 
 dense_2 (Dense)             (None, 1)                 197       
                                                                 
Total params: 1,534,997
Trainable params: 1,534,997
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X, Y, epochs=10)

Epoch 1/10
131/131 [==============================] - 116s 866ms/step - loss: 0.1509 - accuracy: 0.9521
Epoch 2/10
131/131 [==============================] - 140s 1s/step - loss: 0.0252 - accuracy: 0.9928
Epoch 3/10
131/131 [==============================] - 123s 940ms/step - loss: 0.0100 - accuracy: 0.9971
Epoch 4/10
131/131 [==============================] - 117s 894ms/step - loss: 0.0059 - accuracy: 0.9986
Epoch 5/10
131/131 [==============================] - 115s 879ms/step - loss: 0.0026 - accuracy: 0.9995
Epoch 6/10
131/131 [==============================] - 110s 840ms/step - loss: 0.0012 - accuracy: 0.9998
Epoch 7/10
131/131 [==============================] - 102s 780ms/step - loss: 4.8162e-04 - accuracy: 0.9998
Epoch 8/10
131/131 [==============================] - 103s 785ms/step - loss: 9.4739e-05 - accuracy: 1.0000
Epoch 9/10
131/131 [==============================] - 103s 784ms/step - loss: 7.3394e-05 - accuracy: 1.0000
Epoch 10/10
131/131 [==============================] - 

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  pred_text = re.sub('[^a-zA-z0-9\s]', ' ', pred_text)

  test_X = tokenizer.texts_to_sequences([pred_text])
  test_X = pad_sequences(test_X, maxlen=190)

  num = model.predict(test_X)[0]
  label = ["ham", "spam"][round(num[0])]

  prediction = [num, label]
  return prediction

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 0s 296ms/step
[array([3.9927722e-06], dtype=float32), 'ham']


In [ ]:
predict_message("sale today! to stop texts call 98912460324")

1/1 [==============================] - 0s 70ms/step


[array([0.5638242], dtype=float32), 'spam']

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 [==============================] - 0s 68ms/step
You passed the challenge. Great job!
